In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.runnables import RunnableSequence
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
import os

load_dotenv()
api_key_01 = os.getenv("HUGGINGFACE_API_KEY")
api_key_02 = os.getenv("GOOGLE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key_01,
    temperature=0.1 # Lower temperature for better JSON generation
)

model_01 = ChatHuggingFace(llm=llm) 

model_02 = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.1, api_key=api_key_02)  

str_parser = StrOutputParser()

prompt_01 = PromptTemplate(
    template='Generte a tweet about the following topic: {topic}.',
    input_variables=["topic"]
)

prompt_02 = PromptTemplate(
    template='Generte a LinkedIn post description about the following topic: {topic}.',
    input_variables=["topic"]
)

chain = RunnableParallel(
    {"tweet": RunnableSequence(prompt_01 | model_01 | str_parser), "linkedin": RunnableSequence(prompt_02 | model_01 | str_parser)}
)

result = chain.invoke({"topic": "Artificial Intelligence"})

print(result)

{'tweet': '"The future is now: Artificial Intelligence is revolutionizing the way we live, work, and interact. From personalized medicine to intelligent assistants, AI is transforming industries and changing the game. What\'s next? #ArtificialIntelligence #AI #FutureTech"', 'linkedin': 'Here are a few options for a LinkedIn post description about Artificial Intelligence:\n\n**Option 1:**\n"Unlocking the Future of Business: The Power of Artificial Intelligence. As AI continues to transform industries and revolutionize the way we work, it\'s essential to stay ahead of the curve. From predictive analytics to automation, AI is changing the game. Let\'s explore the possibilities and opportunities that AI has to offer. Share your thoughts and experiences with AI in the comments below! #ArtificialIntelligence #AI #FutureOfWork"\n\n**Option 2:**\n"Artificial Intelligence: The Key to Unlocking Human Potential. AI is no longer just a buzzword - it\'s a reality that\'s transforming the way we liv

In [9]:
chain.get_graph().print_ascii()

            +-------------------------------+            
            | Parallel<tweet,linkedin>Input |            
            +-------------------------------+            
                  ***               ***                  
               ***                     ***               
             **                           **             
+----------------+                    +----------------+ 
| PromptTemplate |                    | PromptTemplate | 
+----------------+                    +----------------+ 
          *                                   *          
          *                                   *          
          *                                   *          
+-----------------+                  +-----------------+ 
| ChatHuggingFace |                  | ChatHuggingFace | 
+-----------------+                  +-----------------+ 
          *                                   *          
          *                                   *          
          *   